In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

'''image_classes = np.loadtxt('/home/ap19060/Downloads/hirise ma'+'/labels-map-proj-v3.txt',dtype=str,delimiter='')
df = pd.DataFrame(image_classes,columns=['filename','class'])

x_train, y_train, x_test, y_test = getMNIST()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test  = x_test.reshape((x_test.shape[0], 28, 28, 1))


print("\tN(train)             = ", len(x_train))
print("\tN(test)              = ", len(x_test))
print("\tTest data shape      = ", x_train.shape )
print("\tTrain data shape     = ", x_test.shape )'''

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_classes = np.loadtxt('labels-map-proj-v3.txt',dtype=str,delimiter=' ')
df =pd.DataFrame(image_classes,columns=['filename','class'])
idg = ImageDataGenerator(rescale=1./255.) # normalize
batchsize=10
train_ds = idg.flow_from_dataframe(dataframe=df, directory='map-proj-v3',batch_size=batchsize)

In [ ]:
print(df)

In [ ]:
print(len(train_ds[0][0]))
pixels = []
classes = []
for i in range(len(train_ds[0][0])):
    pixels.append(train_ds[0][0][i])
    classes.append(train_ds[0][1][i])
test = 3
plt.imshow(pixels[test])
print(pixels[test].shape[1])

In [ ]:
#
# Training configuration
#
BatchSize       = batchsize # be careful with this, if it is too big may crash!
Nepochs         = 20
DropoutValue    = 0.6


xpix = pixels[test].shape[0]
ypix = pixels[test].shape[1]
zpix = pixels[test].shape[2] #colour
print(xpix, ypix, zpix)

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(xpix, ypix, zpix)))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.6))
model.add(keras.layers.Dense(10, activation='softmax'))

print("--------------------------------------------------------------------------------------------------------------")
print("\033[92mWill train a convolutional neural network on the MNIST data\033[0m")
print("--------------------------------------------------------------------------------------------------------------\n\n")
print("Input data MNIST")
print("Dropout values       = ", DropoutValue)
print("Leaky relu parameter =  0.1")
print("ValidationSplit      = ", ValidationSplit)
print("BatchSize            = ", BatchSize)
print("Nepochs              = ", Nepochs, "\n")
print("N(train)             = ", len(pixels))
model.summary()

# now specify the loss function - cross entropy
loss_fn = keras.losses.CategoricalCrossentropy()

# now we can train the model to make predictions.
#   Use the ADAM optimiser
#   Specify the metrics to report as accuracy
#   Specify the loss function (see above)
# the fit step specifies the number of training epochs
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
history  = model.fit(train_ds, batch_size=10,epochs=Nepochs)

#